# Sub tests 

In [13]:
import zmq
import pickle
import numpy as np 
import plotly.graph_objs as go
import plotly.subplots as sp

In [14]:
context = zmq.Context()
subscriber = context.socket(zmq.SUB)
subscriber.bind("tcp://127.0.0.1:5556")
subscriber.setsockopt_string(zmq.SUBSCRIBE, "") 

def rcv_msg(): 
    return subscriber.recv_string()

In [15]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
from collections import defaultdict
from IPython.display import display, clear_output
import time

# Initialize subplots
fig = make_subplots(rows=3, cols=1, subplot_titles=('WindVelX', 'WindVelY', 'WindVelZ'))
fig.add_scatter(y=[], mode='lines', name='WindVelX', row=1, col=1)
fig.add_scatter(y=[], mode='lines', name='WindVelY', row=2, col=1)
fig.add_scatter(y=[], mode='lines', name='WindVelZ', row=3, col=1)

received_updates = defaultdict(list)  # Store received updates for each velocity component

update_frequency = 10  # Update plots every 'N' timesteps
iteration_count = 0

plot_title = "t = 0.00"

while True:
    update = rcv_msg()
    component, value = update.split('=')  
    
    component = component.strip()
    if 'time' in component:
        plot_title = 't = {}'.format(value)
    else:
        component = component.replace('(m/s)', '').strip()
        value = float(value.strip()) 
        
    received_updates[component].append(value)  
    
    iteration_count += 1
    
    if iteration_count % update_frequency == 0:
        clear_output(wait=True)
        
        # Update the plots with the received updates
        for i, trace_name in enumerate(['WindVelX', 'WindVelY', 'WindVelZ']):
            fig.data[i].y = received_updates[trace_name]
        
        display(fig)
        fig.update_layout(title=plot_title)
        time.sleep(1)


KeyboardInterrupt: 